In [30]:
import numpy as np
import nltk
from nltk.text import TextCollection
from pymongo import MongoClient
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from numpy import dot
from numpy.linalg import norm

In [31]:
client = MongoClient("localhost")
db=client.crawlerdb_WORK_TFIDF # crawl_database_1 is the db name
collection = db.crawlerdb
result_documents_list_REAL = list(collection.find({})) # find returns cursor, like an iterator in scala
print("We got ", len(result_documents_list_REAL), " Documents")

We got  38525  Documents


In [32]:
result_documents_list = [{'_id': '5c2b861a7faca327135730a6',
  'date': '27.06.2017',
  'title': 'Ein Schwerverletzter und zwei Leichtverletzte bei Auseinandersetzung',
  'location': 'Mitte',
  'text': 'hallo hier bin In der Nacht kam es in Mitte zu einer Auseinandersetzung zwischen mehreren Personen, bei der drei Männer verletzt wurden. Ersten Ermittlungen zufolge sollen sich die drei Verletzten, im Alter von 18, 20 und 21 Jahren, gemeinsam mit Freunden und ihnen Unbekannten gegen Mitternacht am Neptunbrunnen in der Spandauer Straße aufgehalten haben. Plötzlich habe ein Unbekannter eine Glasfasche geworfen, die jedoch niemanden traf. Daraufhin sei es zu einem Handgemenge gekommen, in dessen Verlauf zwei Männer Reizgas versprühten. Die beiden 18 und 20 Jahre alten Männer erlitten leichte Augenreizungen, die zum Teil ambulant von alarmierten Rettungskräften behandelt wurden. Im Zuge der Anzeigenaufnahme erfuhren die Beamten, dass unter der Liebknechtbrücke der 21-Jährige liege, der eine Schnittwunde aufwies. Diese habe er von einem ihm namentlich nicht bekannten Mann im Anschluss an das Handgemenge mit einem Messer zugefügt bekommen. Rettungskräfte brachten den Schwerverletzten zur\n    stationären Behandlung in eine Klinik. Die genauen Hintergründe der Tat sind nun Teil der Ermittlungen, die die Kriminalpolizei der Direktion 3 führt.',
  'number': ['Nr. 1431', '']},
 {'_id': '5c2b861a7faca327135730a7',
  'date': '17.07.2017',
  'title': 'Ein Krankenwagenunfall',
  'location': 'Osten',
  'text': 'hallo Während des Tages kam es in Mitte zu einer Auseinandersetzung zwischen mehreren Personen, bei der drei Männer verletzt wurden. Ersten Ermittlungen zufolge sollen sich die drei Verletzten, im Alter von 18, 20 und 21 Jahren, gemeinsam mit Freunden und ihnen Unbekannten gegen Mitternacht am Neptunbrunnen in der Spandauer Straße aufgehalten haben. Plötzlich habe ein Unbekannter eine Glasfasche geworfen, die jedoch niemanden traf. Daraufhin sei es zu einem Handgemenge gekommen, in dessen Verlauf zwei Männer Reizgas versprühten. Die beiden 18 und 20 Jahre alten Männer erlitten leichte Augenreizungen, die zum Teil ambulant von alarmierten Rettungskräften behandelt wurden. Im Zuge der Anzeigenaufnahme erfuhren die Beamten, dass unter der Liebknechtbrücke der 21-Jährige liege, der eine Schnittwunde aufwies. Diese habe er von einem ihm namentlich nicht bekannten Mann im Anschluss an das Handgemenge mit einem Messer zugefügt bekommen. Rettungskräfte brachten den Schwerverletzten zur\n    stationären Behandlung in eine Klinik. Die genauen Hintergründe der Tat sind nun Teil der Ermittlungen, die die Kriminalpolizei der Direktion 3 führt.',
  'number': ['Nr. 1430', '']},
 {'_id': '5c2b861a7faca327135730a8',
  'date': '16.07.2017',
  'title': 'Feuer in einem Haus',
  'location': 'Westen',
  'text': 'hier Während des Tages kam es in Mitte zu einem Konflikt zwischen mehreren Personen, bei der drei Männer verletzt wurden. Ersten Ermittlungen zufolge sollen sich die drei Verletzten, im Alter von 18, 20 und 21 Jahren, gemeinsam mit Freunden und ihnen Unbekannten gegen Mitternacht am Neptunbrunnen in der Spandauer Straße aufgehalten haben. Plötzlich habe ein Unbekannter eine Glasfasche geworfen, die jedoch niemanden traf. Daraufhin sei es zu einem Handgemenge gekommen, in dessen Verlauf zwei Männer Reizgas versprühten. Die beiden 18 und 20 Jahre alten Männer erlitten leichte Augenreizungen, die zum Teil ambulant von alarmierten Rettungskräften behandelt wurden. Im Zuge der Anzeigenaufnahme erfuhren die Beamten, dass unter der Liebknechtbrücke der 21-Jährige liege, der eine Schnittwunde aufwies. Diese habe er von einem ihm namentlich nicht bekannten Mann im Anschluss an das Handgemenge mit einem Messer zugefügt bekommen. Rettungskräfte brachten den Schwerverletzten zur\n    stationären Behandlung in eine Klinik. Die genauen Hintergründe der Tat sind nun Teil der Ermittlungen, die die Kriminalpolizei der Direktion 3 führt.',
  'number': ['Nr. 1432', '1430']},
 {'_id': '5c2b861a7faca327135730a9',
  'date': '17.07.2017',
  'title': 'Feuer in einem Haus',
  'location': 'Westen',
  'text': 'bin Während des Tages kam es in Mitte zu einem Konflikt zwischen einer Person, bei der 4 Männer verletzt wurden. Ersten Ermittlungen zufolge sollen sich die drei Verletzten, im Alter von 18, 20 und 21 Jahren, gemeinsam mit Freunden und ihnen Unbekannten gegen Mitternacht am Neptunbrunnen in der Spandauer Straße aufgehalten haben. Plötzlich habe ein Unbekannter eine Glasfasche geworfen, die jedoch niemanden traf. Daraufhin sei es zu einem Handgemenge gekommen, in dessen Verlauf zwei Männer Reizgas versprühten. Die beiden 18 und 20 Jahre alten Männer erlitten leichte Augenreizungen, die zum Teil ambulant von alarmierten Rettungskräften behandelt wurden. Im Zuge der Anzeigenaufnahme erfuhren die Beamten, dass unter der Liebknechtbrücke der 21-Jährige liege, der eine Schnittwunde aufwies. Diese habe er von einem ihm namentlich nicht bekannten Mann im Anschluss an das Handgemenge mit einem Messer zugefügt bekommen. Rettungskräfte brachten den Schwerverletzten zur\n    stationären Behandlung in eine Klinik. Die genauen Hintergründe der Tat sind nun Teil der Ermittlungen, die die Kriminalpolizei der Direktion 3 führt.',
  'number': ['Nr. 1428', '']},
 {'_id': '5c2b861a7faca327135730aa',
  'date': '15.07.2017',
  'title': 'Feuer in einem Haus',
  'location': 'Norden',
  'text': 'ich Während des Tages kam es in Mitte zu einem Konflikt zwischen einer Person, bei der 4 Männer verwunded wurden. Ersten Ermittlungen zufolge sollen sich die drei Verletzten, im Alter von 18, 20 und 21 Jahren, gemeinsam mit Freunden und ihnen Unbekannten gegen Mitternacht am Neptunbrunnen in der Spandauer Straße aufgehalten haben. Plötzlich habe ein Unbekannter eine Glasfasche geworfen, die jedoch niemanden traf. Daraufhin sei es zu einem Handgemenge gekommen, in dessen Verlauf zwei Männer Reizgas versprühten. Die beiden 18 und 20 Jahre alten Männer erlitten leichte Augenreizungen, die zum Teil ambulant von alarmierten Rettungskräften behandelt wurden. Im Zuge der Anzeigenaufnahme erfuhren die Beamten, dass unter der Liebknechtbrücke der 21-Jährige liege, der eine Schnittwunde aufwies. Diese habe er von einem ihm namentlich nicht bekannten Mann im Anschluss an das Handgemenge mit einem Messer zugefügt bekommen. Rettungskräfte brachten den Schwerverletzten zur\n    stationären Behandlung in eine Klinik. Die genauen Hintergründe der Tat sind nun Teil der Ermittlungen, die die Kriminalpolizei der Direktion 3 führt.',
  'number': ['Nr. 1427', '']}]

In [33]:
#result_document_text_list = [a["text"] for a in result_documents_list_REAL] # wir müssen erst result_documents_list["text"] "entpacken" von cursor
result_document_text_list = []
for i in range(len(result_documents_list_REAL)):
    try:
        result_document_text_list.append(result_documents_list_REAL[i]["text"]) # Fehler: KeyError: muss abgefangen werden
    except KeyError:
        pass
print("We still hab")

In der Nacht kam es in Mitte zu einer Auseinandersetzung zwischen mehreren Personen, bei der drei Männer verletzt wurden. Ersten Ermittlungen zufolge sollen sich die drei Verletzten, im Alter von 18, 20 und 21 Jahren, gemeinsam mit Freunden und ihnen Unbekannten gegen Mitternacht am Neptunbrunnen in der Spandauer Straße aufgehalten haben. Plötzlich habe ein Unbekannter eine Glasfasche geworfen, die jedoch niemanden traf. Daraufhin sei es zu einem Handgemenge gekommen, in dessen Verlauf zwei Männer Reizgas versprühten. Die beiden 18 und 20 Jahre alten Männer erlitten leichte Augenreizungen, die zum Teil ambulant von alarmierten Rettungskräften behandelt wurden. Im Zuge der Anzeigenaufnahme erfuhren die Beamten, dass unter der Liebknechtbrücke der 21-Jährige liege, der eine Schnittwunde aufwies. Diese habe er von einem ihm namentlich nicht bekannten Mann im Anschluss an das Handgemenge mit einem Messer zugefügt bekommen. Rettungskräfte brachten den Schwerverletzten zur
    stationären Be

'Ein 22-Jähriger zeigte heute bei der Polizei an, dass er gestern Nachmittag in Oberschöneweide von einem Unbekannten beleidigt und bedroht worden sei. Des Weiteren hätte der Mann eine Glasflasche in seine Richtung geworfen, ihn mit einem Messer bedroht und ihn geschubst.\n    Nach Angaben des 22-jährigen mit türkischem Migrationshintergrund, sei er gestern gegen 17 Uhr mit der Straßenbahn der Linie 67 in Richtung Schöneweide unterwegs gewesen. An der Haltestelle Wilhelminenhofstraße / Edisonstraße habe er einen anderen Fahrgast gebeten, seine laut gehörte Musik leiser zu stellen. Daraufhin hätte der Unbekannte ihn sofort laut angeschrien und ihn fremdenfeindlich beleidigt. Daraufhin sei der 22-Jährige von dem aggressiven Mann weggegangen und zwei Stationen später, am S-Bahnhof Schöneweide, ausgestiegen. Der Unbekannte sei dann ebenfalls ausgestiegen und wenig später habe der 22-Jährige hinter sich eine Glasflasche zerschellen hören und Glassplitter gespürt. Der Unbekannte sei dann von

In [6]:
search_term = ["hallo hier bin ich"]

In [7]:
tfidf = TfidfVectorizer()
tfidf_values = tfidf.fit_transform(result_document_text_list)
pd.DataFrame(tfidf_values.todense(),columns=tfidf.get_feature_names())
print("finished")

finished


In [8]:
tfidf_search_term = tfidf.transform(search_term)
#tfidf_search_term.todense()

In [9]:
for i in range(len(result_document_text_list)):
    tfidf_values_tmp = np.array(tfidf_values.todense()[i]).reshape(-1)
    tfidf_search_term_tmp = np.array(tfidf_search_term.todense()).reshape(-1)
    
    first = np.dot(tfidf_values_tmp, tfidf_search_term_tmp)
    second = (norm(tfidf_values_tmp)*norm(tfidf_search_term_tmp))
    cos_sim = first * 1.0/second
    print(cos_sim)

0.11870806104324305
0.04134612329345565
0.041085180497238465
0.041233361834068095
0.06296299609272991
